In [114]:
import sqlite3
import pandas as pd
import dfformat

with sqlite3.connect("netkeiba.sqlite") as conn:
    df_original = pd.read_sql_query("SELECT * FROM horse", conn)

In [159]:
df = dfformat.format(df_original)

In [138]:
df_original.T

,0,1,2,3,4,5,6,7,8,9,...,686205,686206,686207,686208,686209,686210,686211,686212,686213,686214
id,1,2,3,4,5,6,7,8,9,10,...,686206,686207,686208,686209,686210,686211,686212,686213,686214,686215
result,1,2,3,4,5,6,7,8,9,10,...,3,4,5,6,7,8,9,10,11,12
gate,1,3,2,4,6,2,7,1,4,8,...,5,1,8,8,7,6,3,4,2,5
horse_no,2,5,3,7,12,4,14,1,8,15,...,5,1,12,11,9,8,3,4,2,6
name,メジロアリエル,ヒロアンジェロ,キャスタスペルミー,デルマベガ,メジロフローラ,チャームベル,アメリカンヒロイン,エイトシード,フォンテーヌ,コスモドリーミン,...,マリノソフィア,ダディーズトリップ,ダノングレーター,ピノクル,ルージュメサージュ,ビートザウイングス,ルヴェルソー,メイショウボサツ,バラードインミラノ,タイフォン
sex,牝,牝,牝,牝,牝,牝,牝,牝,牝,牝,...,牝,牡,セ,牡,牝,セ,牝,牡,牡,牡
age,3,3,3,3,3,3,3,3,3,3,...,4,5,5,5,3,4,4,4,3,3
penalty,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,51.0,...,55.0,57.0,57.0,57.0,54.0,57.0,55.0,57.0,56.0,56.0
jockey,吉田豊,蛯名,田中勝,吉田隼,石橋脩,菊沢,小林淳,村田,嘉藤,▲佐藤聖,...,黛,菱田,坂井,富田,鮫島駿,秋山稔,吉田隼,小沢,斎藤,西村淳
time,1:13.9,1:13.9,1:14.2,1:14.3,1:14.5,1:14.9,1:15.0,1:15.2,1:15.3,1:15.4,...,2:01.0,2:01.2,2:01.2,2:01.3,2:01.5,2:01.5,2:01.5,2:02.0,2:02.0,2:02.7


In [ ]:
.groupby(["year", "place_code", "hold_num", "day_num", "race_num"])["id"].count(df_original)

In [ ]:
df_original.query("2008 <= year <= 2017").groupby(["year", "place_code", "hold_num", "day_num", "race_num"])["id"].count()

In [ ]:
df_original.query("2018 <= year <= 2020").groupby(["year", "place_code", "hold_num", "day_num", "race_num"])["id"].count()

In [ ]:
df_original.query("2021 <= year").groupby(["year", "place_code", "hold_num", "day_num", "race_num"])["id"].count()

In [ ]:
df.T

In [198]:
from npfeat import (ave, calc_ave_time, diff, interval, same_count, time,
                    yield_calculated)

ave_time = calc_ave_time(df)
hist_pattern = [1, 2, 3, 4, 5, 10, 999999]
feat_pattern = {
    "horse_interval": interval,
    "horse_place": same_count("place_code"),
    "horse_odds": ave("odds"),
    "horse_pop": ave("pop"),
    "horse_result": ave("result"),
    "horse_jockey": same_count("jockey"),
    "horse_penalty": ave("penalty"),
    "horse_distance": diff("distance"),
    "horse_weather": same_count("weather"),
    "horse_fc": same_count("field_condition"),
    "horse_time": time(ave_time),
    "horse_margin": ave("margin"),
    "horse_corner3": ave("corner3"),
    "horse_corner4": ave("corner4"),
    "horse_last3f": ave("last3f"),
    "horse_weight": ave("weight"),
    "horse_wc": ave("weight_change"),
    "horse_prize": ave("prize"),
}

In [ ]:
with sqlite3.connect("feature2.sqlite") as conn:
    for name, hist_df in yield_calculated(df, hist_pattern, feat_pattern):
        print("\r"+str(name),end="")
        hist_df.to_sql("horse", conn, if_exists="append", index_label="id")

In [1]:
import sqlite3
import pandas as pd
with sqlite3.connect("feature2.sqlite") as conn:
    df_feat = pd.read_sql_query("SELECT * FROM horse", conn).sort_values("id").reset_index()
df_feat.T

,0,1,2,3,4,5,6,7,8,9,...,686205,686206,686207,686208,686209,686210,686211,686212,686213,686214
index,592443,455232,127095,368636,593372,341793,27521,69286,478395,180084,...,554658,331208,332073,465384,646271,461720,645895,587086,440311,311778
id,1,2,3,4,5,6,7,8,9,10,...,686206,686207,686208,686209,686210,686211,686212,686213,686214,686215
result,1,2,3,4,5,6,7,8,9,10,...,3,4,5,6,7,8,9,10,11,12
gate,1,3,2,4,6,2,7,1,4,8,...,5,1,8,8,7,6,3,4,2,5
horse_no,2,5,3,7,12,4,14,1,8,15,...,5,1,12,11,9,8,3,4,2,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
horse_prize_3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,403.333333,176.666667,0.0,153.333333,423.333333,316.666667,203.333333,126.666667,110.0,406.666667
horse_prize_4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,340.0,132.5,57.5,305.0,330.25,237.5,302.5,95.0,82.5,331.5
horse_prize_5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,392.0,226.0,46.0,304.0,294.2,244.0,288.0,152.0,92.0,265.2
horse_prize_10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,295.0,194.0,38.0,258.0,294.2,178.6,320.0,189.0,105.714286,183.6


In [31]:
def score(x):
    scores = [20, 8, 5, 3, 2]
    result = x["result"]
    if result <= 5:
        return scores[result-1]
    else:
        return 0

score = df_feat.apply(score, axis=1)

In [32]:
df_feat["score"] = score

In [34]:
df_feat.head().T

,0,1,2,3,4
index,592443,455232,127095,368636,593372
id,1,2,3,4,5
result,1,2,3,4,5
gate,1,3,2,4,6
horse_no,2,5,3,7,12
...,...,...,...,...,...
horse_prize_4,NaN,NaN,NaN,NaN,NaN
horse_prize_5,NaN,NaN,NaN,NaN,NaN
horse_prize_10,NaN,NaN,NaN,NaN,NaN
horse_prize_999999,NaN,NaN,NaN,NaN,NaN


In [233]:
import lightgbm as lgb
from lightgbm import Dataset
noneed_columns =[
    "index", "id", "result", "time", "margin", "pop", "odds", "last3f", \
    "weight", "weight_change", "corner", "corner3", "corner4", \
    "year", "hold_num", "race_num", "day_num", "race_date", "race_name", \
    "start_time", "prize1", "prize2", "prize3", "prize4", "prize5", "prize"
]
race_columns = ["year", "place_code", "hold_num", "day_num", "race_num"]

def prepare_dataset(df, target):
    query = df.groupby(race_columns)["id"].count().values.tolist()
    x = df.drop(columns=noneed_columns)
    y = x.pop(target)
    dataset = Dataset(x, y, group=query)
    return dataset

train = prepare_dataset(df_feat.query("'2008-01-01' <= race_date <= '2017-12-31'"), "score")
valid = prepare_dataset(df_feat.query("'2018-01-01' <= race_date <= '2020-12-31'"), "score")
test = prepare_dataset(df_feat.query("'2021-01-01' <= race_date <= '2021-12-31'"), "score")
columns = train.data.columns

In [234]:
train.data.shape, train.label.shape, valid.data.shape, valid.label.shape, test.data.shape, test.label.shape

((497689, 143), (497689,), (143161, 143), (143161,), (45365, 143), (45365,))

In [81]:
lgb_params = {
    "objective": "lambdarank",
    "metric": "ndcg",
    "lambdarank_truncation_level": 10,
    "ndcg_eval_at": [1,2,3,4,5],
    "boosting_type": "gbdt",
    "learning_rate": 0.01,
}
callbacks = [
    lgb.log_evaluation(10),
    lgb.early_stopping(50),
]
rank_model = lgb.train(
   lgb_params,
   train,
   num_boost_round=10000,
   valid_sets=valid,
   callbacks=callbacks,
)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.929514 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 21154
[LightGBM] [Info] Number of data points in the train set: 497689, number of used features: 143
Training until validation scores don't improve for 50 rounds
[10]	valid_0's ndcg@1: 0.279785	valid_0's ndcg@2: 0.369571	valid_0's ndcg@3: 0.432687	valid_0's ndcg@4: 0.475089	valid_0's ndcg@5: 0.506035
[20]	valid_0's ndcg@1: 0.279656	valid_0's ndcg@2: 0.371237	valid_0's ndcg@3: 0.434667	valid_0's ndcg@4: 0.475936	valid_0's ndcg@5: 0.507443
[30]	valid_0's ndcg@1: 0.27972	valid_0's ndcg@2: 0.370818	valid_0's ndcg@3: 0.435391	valid_0's ndcg@4: 0.476441	valid_0's ndcg@5: 0.507075
[40]	valid_0's ndcg@1: 0.280983	valid_0's ndcg@2: 0.371984	valid_0's ndcg@3: 0.435206	valid_0's ndcg@4: 0.477279	valid_0's ndcg@5: 0.508281
[50]	valid_0's ndcg@1: 0.279172	valid_0's ndcg@2: 0.370701	valid_0's ndcg@3: 0.433839

In [80]:
importance = pd.Series(rank_model.feature_importance(importance_type='gain'), index=columns).sort_values(ascending=False)
display(importance[:20])

horse_odds_1            11755.827805
horse_result_1           6644.487030
horse_odds_5              929.484985
horse_pop_2               572.314026
horse_odds_4              306.071999
horse_margin_999999       294.070999
jockey                    282.529003
horse_prize_999999        262.540985
horse_result_10           168.968002
cos_starttime             123.158997
horse_odds_3               73.503403
horse_time_1               71.218300
horse_last3f_3             66.311096
horse_margin_1             64.499901
horse_corner3_3            63.307899
horse_prize_2              62.206402
field                      59.672100
horse_time_2                0.000000
horse_fc_5                  0.000000
horse_weather_999999        0.000000
dtype: float64

In [93]:
rank_pred = rank_model.predict(test.data, num_iteration=rank_model.best_iteration)

In [87]:
noneed_columns =[
    "index", "id", "result", "time", "margin", "pop", "odds", "last3f", \
    "weight", "weight_change", "corner", "corner3", "corner4", \
    "year", "hold_num", "race_num", "day_num", "race_date", "race_name", \
    "start_time", "prize1", "prize2", "prize3", "prize4", "prize5", "score"
]
def prepare_dataset(df, target):
    query = df.groupby(race_columns)["id"].count().values.tolist()
    x = df.drop(columns=noneed_columns)
    y = x.pop(target)
    dataset = Dataset(x, y, group=query)
    return dataset
train = prepare_dataset(df_feat.query("'2008-01-01' <= race_date <= '2017-12-31'"), "prize")
valid = prepare_dataset(df_feat.query("'2018-01-01' <= race_date <= '2020-12-31'"), "prize")
test = prepare_dataset(df_feat.query("'2021-01-01' <= race_date <= '2021-12-31'"), "prize")
columns = train.data.columns
lgb_params = {
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'metric': 'rmse',
    'learning_rate': 0.01
}
callbacks = [
    lgb.log_evaluation(10),
    lgb.early_stopping(50),
]
reg_model = lgb.train(
   lgb_params,
   train,
   num_boost_round=10000,
   valid_sets=valid,
   callbacks=callbacks,
)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.962087 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 21154
[LightGBM] [Info] Number of data points in the train set: 497689, number of used features: 143
[LightGBM] [Info] Start training from score 135.972320
Training until validation scores don't improve for 50 rounds
[10]	valid_0's rmse: 486.949
[20]	valid_0's rmse: 478.441
[30]	valid_0's rmse: 471.492
[40]	valid_0's rmse: 465.58
[50]	valid_0's rmse: 460.721
[60]	valid_0's rmse: 456.666
[70]	valid_0's rmse: 453.113
[80]	valid_0's rmse: 449.885
[90]	valid_0's rmse: 447.07
[100]	valid_0's rmse: 444.551
[110]	valid_0's rmse: 442.295
[120]	valid_0's rmse: 440.336
[130]	valid_0's rmse: 438.66
[140]	valid_0's rmse: 437.345
[150]	valid_0's rmse: 436.072
[160]	valid_0's rmse: 434.849
[170]	valid_0's rmse: 433.72
[180]	valid_0's rmse: 433.047
[190]	valid_0's rmse: 432.186
[200]	valid_0's rmse: 431.508
[2

In [90]:
importance = pd.Series(reg_model.feature_importance(importance_type='gain'), index=columns).sort_values(ascending=False)
display(importance[:50])

horse_prize_10           3.881081e+11
horse_prize_5            1.669146e+11
horse_prize_3            1.601791e+11
horse_prize_999999       1.239449e+11
distance                 7.174409e+10
horse_prize_1            6.993384e+10
horse_odds_1             6.687102e+10
horse_prize_4            6.018093e+10
horse_odds_2             4.782102e+10
horse_prize_2            4.486344e+10
horse_interval_1         3.634264e+10
horse_interval_2         3.605684e+10
gate                     3.593076e+10
jockey                   3.496315e+10
horse_time_1             3.026646e+10
horse_odds_3             2.536316e+10
name                     2.126868e+10
horse_last3f_1           1.993408e+10
horse_last3f_2           1.777454e+10
horse_time_4             1.720025e+10
horse_pop_5              1.600558e+10
horse_time_2             1.495845e+10
horse_result_1           1.478711e+10
cos_starttime            1.437372e+10
horse_pop_10             1.400526e+10
horse_margin_1           1.395687e+10
horse_interv

In [91]:
reg_pred = reg_model.predict(test.data, num_iteration=reg_model.best_iteration)

In [102]:
pd.DataFrame({"result": test.label, "rank": rank_pred*10000, "reg": reg_pred,}).head()

,result,rank,reg
640850,20,105.555291,173.621978
640851,8,136.776910,234.568790
640852,5,61.508450,105.347376
640853,3,53.468676,74.312491
640854,2,-59.195862,67.174635


In [121]:
arima_horses = [
    [None,1,1,"ペルシアンナイト","牡",7,57.0,"Ｃデムーロ",None,None,12,51.0,None,None,"栗東池江",None,None,"有馬記念","15:25","芝",2500,"右","晴","良","サラ系３歳以上オープン",30000,12000,7500,4500,3000,2021,6,5,8,11,"12月26日(日)"],
    [None,1,2,"パンサラッサ","牡",4,57.0,"菱田",None,None,9,32.3,None,None,"栗東矢作",None,None,"有馬記念","15:25","芝",2500,"右","晴","良","サラ系３歳以上オープン",30000,12000,7500,4500,3000,2021,6,5,8,11,"12月26日(日)"],
    [None,2,3,"モズベッロ","牡",5,57.0,"池添",None,None,14,92.6,None,None,"栗東森田",None,None,"有馬記念","15:25","芝",2500,"右","晴","良","サラ系３歳以上オープン",30000,12000,7500,4500,3000,2021,6,5,8,11,"12月26日(日)"],
    [None,2,4,"メロディーレーン","牝",5,55.0,"岩田望",None,None,14,71.0,None,None,"栗東森田",None,None,"有馬記念","15:25","芝",2500,"右","晴","良","サラ系３歳以上オープン",30000,12000,7500,4500,3000,2021,6,5,8,11,"12月26日(日)"],
    [None,3,5,"ディープボンド","牡",4,57.0,"和田竜",None,None,5,18.2,None,None,"栗東大久保",None,None,"有馬記念","15:25","芝",2500,"右","晴","良","サラ系３歳以上オープン",30000,12000,7500,4500,3000,2021,6,5,8,11,"12月26日(日)"],
    [None,3,6,"ウインキートス","牝",4,55.0,"丹内",None,None,11,48.9,None,None,"美浦宗像",None,None,"有馬記念","15:25","芝",2500,"右","晴","良","サラ系３歳以上オープン",30000,12000,7500,4500,3000,2021,6,5,8,11,"12月26日(日)"],
    [None,4,7,"クロノジェネシス","牝",5,55.0,"ルメール",None,None,2,3.2,None,None,"栗東斉藤崇",None,None,"有馬記念","15:25","芝",2500,"右","晴","良","サラ系３歳以上オープン",30000,12000,7500,4500,3000,2021,6,5,8,11,"12月26日(日)"],
    [None,4,8,"ユーキャンスマイル","牡",6,57.0,"藤岡佑",None,None,15,95.3,None,None,"栗東友道",None,None,"有馬記念","15:25","芝",2500,"右","晴","良","サラ系３歳以上オープン",30000,12000,7500,4500,3000,2021,6,5,8,11,"12月26日(日)"],
    [None,5,9,"ステラヴェローチェ","牡",3,55.0,"Ｍデムーロ",None,None,3,9.0,None,None,"栗東須貝",None,None,"有馬記念","15:25","芝",2500,"右","晴","良","サラ系３歳以上オープン",30000,12000,7500,4500,3000,2021,6,5,8,11,"12月26日(日)"],
    [None,5,10,"エフフォーリア","牡",3,55.0,"横山武",None,None,1,2.5,None,None,"美浦鹿戸",None,None,"有馬記念","15:25","芝",2500,"右","晴","良","サラ系３歳以上オープン",30000,12000,7500,4500,3000,2021,6,5,8,11,"12月26日(日)"],
    [None,6,11,"アリストテレス","牡",4,57.0,"武豊",None,None,8,27.2,None,None,"栗東音無",None,None,"有馬記念","15:25","芝",2500,"右","晴","良","サラ系３歳以上オープン",30000,12000,7500,4500,3000,2021,6,5,8,11,"12月26日(日)"],
    [None,6,12,"シャドウディーヴァ","牝",5,55.0,"横山典",None,None,16,95.4,None,None,"美浦斎藤誠",None,None,"有馬記念","15:25","芝",2500,"右","晴","良","サラ系３歳以上オープン",30000,12000,7500,4500,3000,2021,6,5,8,11,"12月26日(日)"],
    [None,7,13,"アカイイト","牝",4,55.0,"幸",None,None,7,20.8,None,None,"栗東中竹",None,None,"有馬記念","15:25","芝",2500,"右","晴","良","サラ系３歳以上オープン",30000,12000,7500,4500,3000,2021,6,5,8,11,"12月26日(日)"],
    [None,7,14,"アサマノイタズラ","牡",3,55.0,"田辺",None,None,10,34.9,None,None,"美浦手塚",None,None,"有馬記念","15:25","芝",2500,"右","晴","良","サラ系３歳以上オープン",30000,12000,7500,4500,3000,2021,6,5,8,11,"12月26日(日)"],
    [None,8,15,"キセキ","牡",7,57.0,"松山",None,None,6,20.2,None,None,"栗東辻野",None,None,"有馬記念","15:25","芝",2500,"右","晴","良","サラ系３歳以上オープン",30000,12000,7500,4500,3000,2021,6,5,8,11,"12月26日(日)"],
    [None,8,16,"タイトルホルダー","牡",3,55.0,"横山和",None,None,4,9.0,None,None,"美浦栗田",None,None,"有馬記念","15:25","芝",2500,"右","晴","良","サラ系３歳以上オープン",30000,12000,7500,4500,3000,2021,6,5,8,11,"12月26日(日)"]
]
df_arima = pd.DataFrame(arima_horses, columns=netkeiba.COLUMNS)
df_arima.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
result,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
gate,1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8
horse_no,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
name,ペルシアンナイト,パンサラッサ,モズベッロ,メロディーレーン,ディープボンド,ウインキートス,クロノジェネシス,ユーキャンスマイル,ステラヴェローチェ,エフフォーリア,アリストテレス,シャドウディーヴァ,アカイイト,アサマノイタズラ,キセキ,タイトルホルダー
sex,牡,牡,牡,牝,牡,牝,牝,牡,牡,牡,牡,牝,牝,牡,牡,牡
age,7,4,5,5,4,4,5,6,3,3,4,5,4,3,7,3
penalty,57.0,57.0,57.0,55.0,57.0,55.0,55.0,57.0,55.0,55.0,57.0,55.0,55.0,55.0,57.0,55.0
jockey,Ｃデムーロ,菱田,池添,岩田望,和田竜,丹内,ルメール,藤岡佑,Ｍデムーロ,横山武,武豊,横山典,幸,田辺,松山,横山和
time,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
margin,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [125]:
df.head().T

,0,1,2,3,4
id,1,2,3,4,5
result,1,2,3,4,5
gate,1,3,2,4,6
horse_no,2,5,3,7,12
name,61164,47193,13164,38001,61248
sex,1,1,1,1,1
age,3,3,3,3,3
penalty,54.0,54.0,54.0,54.0,54.0
jockey,532,939,828,533,868
time,73.9,73.9,74.2,74.3,74.5


In [142]:
df_original.shape, df_arima.shape

((686215, 37), (16, 37))

In [281]:
df_enarima = df_arima.copy()
for column in ["name", "sex", "jockey", "barn", "turn", "weather", "field", "field_condition", "race_condition"]:
    encodeds = []
    for value in df_arima[column].values:
        encoded_values = df[(df_original[column] == value)][column].values
        if encoded_values.any():
            encoded = encoded_values[0]
        else:
            encoded = max(df[column].unique()) + 1
        encodeds.append(encoded)
    df_enarima[column] = pd.Series(encodeds)
df_enarima["race_date"] = dfformat.s_racedate(df_enarima)
df_enarima["cos_racedate"] = dfformat.s_cosracedate(df_enarima["race_date"])
df_enarima["cos_starttime"] = dfformat.s_cosstarttime(df_enarima)
df_enarima["corner3"] = np.nan
df_enarima["corner4"] = np.nan
df_enarima.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
result,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
gate,1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8
horse_no,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
name,53839,46372,61827,61572,37443,5491,15883,63264,28744,7953,2991,24456,682,1243,12740,32130
sex,2,2,2,1,2,1,1,2,2,2,2,1,1,2,2,2
age,7,4,5,5,4,4,5,6,3,3,4,5,4,3,7,3
penalty,57.0,57.0,57.0,55.0,57.0,55.0,55.0,57.0,55.0,55.0,57.0,55.0,55.0,55.0,57.0,55.0
jockey,1060,918,798,649,539,449,1060,929,1060,770,783,768,679,837,722,769
time,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
margin,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [282]:
df_arima.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
result,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
gate,1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8
horse_no,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
name,ペルシアンナイト,パンサラッサ,モズベッロ,メロディーレーン,ディープボンド,ウインキートス,クロノジェネシス,ユーキャンスマイル,ステラヴェローチェ,エフフォーリア,アリストテレス,シャドウディーヴァ,アカイイト,アサマノイタズラ,キセキ,タイトルホルダー
sex,牡,牡,牡,牝,牡,牝,牝,牡,牡,牡,牡,牝,牝,牡,牡,牡
age,7,4,5,5,4,4,5,6,3,3,4,5,4,3,7,3
penalty,57.0,57.0,57.0,55.0,57.0,55.0,55.0,57.0,55.0,55.0,57.0,55.0,55.0,55.0,57.0,55.0
jockey,Ｃデムーロ,菱田,池添,岩田望,和田竜,丹内,ルメール,藤岡佑,Ｍデムーロ,横山武,武豊,横山典,幸,田辺,松山,横山和
time,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
margin,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [287]:
def search_history(name, hist_pattern, feat_pattern):
    with sqlite3.connect("feature2.sqlite") as conn:
        hist = pd.read_sql_query(f"SELECT * FROM horse WHERE name=={name}", conn)
    hist["race_date"] = pd.to_datetime(hist["race_date"])
    hist = hist.loc[:, :'corner4']
    columns = hist.columns.tolist()
    index = lambda x: columns.index(x)
    row_arima = df_enarima[df_enarima["name"] == name]
    hist = pd.concat([hist, row_arima])
    a = hist.values
    funcs = list(feat_pattern.values())
    np_df = X_result(funcs, hist_pattern, a, index)
    past_columns = [f"{col}_{x}" for col in list(feat_pattern.keys()) for x in hist_pattern]
    hist_df = pd.DataFrame(np.concatenate([hist, np_df], axis=1), columns=columns+past_columns)
    hist_df = hist_df.set_index("id")
    return name, hist_df.tail(1)

In [288]:
from npfeat import X_result

df_arimafeat = pd.DataFrame()
for name in df_enarima["name"].unique():
    name, df_f = search_history(name, hist_pattern, feat_pattern)
    print(name, df_f.shape)
    df_arimafeat = pd.concat([df_arimafeat, df_f])
df_arimafeat.T

53839 (1, 167)
46372 (1, 167)
61827 (1, 167)
61572 (1, 167)
37443 (1, 167)
5491 (1, 167)
15883 (1, 167)
63264 (1, 167)
28744 (1, 167)
7953 (1, 167)
2991 (1, 167)
24456 (1, 167)
682 (1, 167)
1243 (1, 167)
12740 (1, 167)
32130 (1, 167)


id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
result,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
gate,1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8
horse_no,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
name,53839,46372,61827,61572,37443,5491,15883,63264,28744,7953,2991,24456,682,1243,12740,32130
sex,2,2,2,1,2,1,1,2,2,2,2,1,1,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
horse_prize_3,933.333333,2233.333333,0.0,606.666667,4233.333333,900.0,16266.666667,0.0,4066.666667,11333.333333,900.0,1833.333333,4106.666667,1800.0,1066.666667,4000.0
horse_prize_4,855.0,1915.0,0.0,455.0,3625.0,2100.0,13550.0,0.0,3750.0,9450.0,1250.0,1600.0,3262.5,1350.0,967.5,4100.0
horse_prize_5,684.0,1532.0,1080.0,364.0,3062.0,1680.0,12080.0,540.0,3076.0,7762.0,1000.0,1352.0,2702.0,1520.0,774.0,4360.0
horse_prize_10,711.0,1131.0,982.0,437.0,2311.0,1378.0,8035.0,1256.0,2772.5,6585.0,1856.0,1048.0,1600.0,1055.0,1407.0,2517.0


In [290]:
noneed_columns = [
    "result", "time", "margin", "pop", "odds", "last3f", \
    "weight", "weight_change", "corner", "corner3", "corner4", \
    "year", "hold_num", "race_num", "day_num", "race_date", "race_name", \
    "start_time", "prize1", "prize2", "prize3", "prize4", "prize5", "prize"
]
df_arimax = df_arimafeat.drop(columns=noneed_columns)
len(df_arimafeat.drop(columns=noneed_columns).columns)

143

In [273]:
df_arimafeat.drop(columns=noneed_columns).columns.values

array(['gate', 'horse_no', 'name', 'sex', 'age', 'penalty', 'jockey',
       'barn', 'field', 'distance', 'turn', 'weather', 'field_condition',
       'race_condition', 'place_code', 'cos_racedate', 'cos_starttime',
       'horse_interval_1', 'horse_interval_1', 'horse_interval_2',
       'horse_interval_3', 'horse_interval_4', 'horse_interval_5',
       'horse_interval_10', 'horse_interval_999999', 'horse_place_1',
       'horse_place_2', 'horse_place_3', 'horse_place_4', 'horse_place_5',
       'horse_place_10', 'horse_place_999999', 'horse_odds_1',
       'horse_odds_2', 'horse_odds_3', 'horse_odds_4', 'horse_odds_5',
       'horse_odds_10', 'horse_odds_999999', 'horse_pop_1', 'horse_pop_2',
       'horse_pop_3', 'horse_pop_4', 'horse_pop_5', 'horse_pop_10',
       'horse_pop_999999', 'horse_result_1', 'horse_result_2',
       'horse_result_3', 'horse_result_4', 'horse_result_5',
       'horse_result_10', 'horse_result_999999', 'horse_jockey_1',
       'horse_jockey_2', 'horse_jocke

In [271]:
len(train.data.columns)

143

In [274]:
train.data.columns.values

array(['gate', 'horse_no', 'name', 'sex', 'age', 'penalty', 'jockey',
       'barn', 'field', 'distance', 'turn', 'weather', 'field_condition',
       'race_condition', 'place_code', 'cos_racedate', 'cos_starttime',
       'horse_interval_1', 'horse_interval_2', 'horse_interval_3',
       'horse_interval_4', 'horse_interval_5', 'horse_interval_10',
       'horse_interval_999999', 'horse_place_1', 'horse_place_2',
       'horse_place_3', 'horse_place_4', 'horse_place_5',
       'horse_place_10', 'horse_place_999999', 'horse_odds_1',
       'horse_odds_2', 'horse_odds_3', 'horse_odds_4', 'horse_odds_5',
       'horse_odds_10', 'horse_odds_999999', 'horse_pop_1', 'horse_pop_2',
       'horse_pop_3', 'horse_pop_4', 'horse_pop_5', 'horse_pop_10',
       'horse_pop_999999', 'horse_result_1', 'horse_result_2',
       'horse_result_3', 'horse_result_4', 'horse_result_5',
       'horse_result_10', 'horse_result_999999', 'horse_jockey_1',
       'horse_jockey_2', 'horse_jockey_3', 'horse_jockey_

In [292]:
reg_pred = reg_model.predict(df_arimax.values, num_iteration=reg_model.best_iteration)

In [307]:
reg_predn = reg_pred / sum(reg_pred) * 100
reg_predn

array([ 2.61357831,  3.83729957,  1.94962106,  1.2280808 ,  7.98351524,
        3.36383216, 21.55162215,  1.60217404,  9.27285544, 27.10522903,
        4.0725833 ,  1.99742318,  4.94200878,  2.30415204,  2.14953167,
        4.02649322])

In [293]:
rank_pred = rank_model.predict(df_arimax.values, num_iteration=rank_model.best_iteration)

In [306]:
rank_predplus = rank_pred - min(rank_pred)
rank_predn = rank_predplus / sum(rank_predplus) * 100
rank_predn

array([ 8.45123966,  7.83351991,  0.        ,  7.83351991, 12.07223112,
        6.2910531 , 10.19984179,  0.        , 10.57023128,  7.83351991,
        3.95412531,  3.95412531,  6.07733655,  4.45130173,  2.64443452,
        7.83351991])

In [325]:
df_arima_pred = df_arima.copy().loc[:, :"barn"].drop(columns=["time", "margin", "last3f", "corner"])
df_arima_pred["reg"] = reg_predn
df_arima_pred["rank"] = rank_predn
df_arima_pred["pred_ave"] = (reg_predn + rank_predn) / 2
df_arima_pred.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
result,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
gate,1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8
horse_no,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
name,ペルシアンナイト,パンサラッサ,モズベッロ,メロディーレーン,ディープボンド,ウインキートス,クロノジェネシス,ユーキャンスマイル,ステラヴェローチェ,エフフォーリア,アリストテレス,シャドウディーヴァ,アカイイト,アサマノイタズラ,キセキ,タイトルホルダー
sex,牡,牡,牡,牝,牡,牝,牝,牡,牡,牡,牡,牝,牝,牡,牡,牡
age,7,4,5,5,4,4,5,6,3,3,4,5,4,3,7,3
penalty,57.0,57.0,57.0,55.0,57.0,55.0,55.0,57.0,55.0,55.0,57.0,55.0,55.0,55.0,57.0,55.0
jockey,Ｃデムーロ,菱田,池添,岩田望,和田竜,丹内,ルメール,藤岡佑,Ｍデムーロ,横山武,武豊,横山典,幸,田辺,松山,横山和
pop,12,9,14,14,5,11,2,15,3,1,8,16,7,10,6,4
odds,51.0,32.3,92.6,71.0,18.2,48.9,3.2,95.3,9.0,2.5,27.2,95.4,20.8,34.9,20.2,9.0


In [355]:
arima_preds = df_arima_pred[["horse_no", "name", "pred_ave", "rank", "reg", "odds"]]

In [356]:
arima_preds.sort_values("pred_ave", ascending=False)

,horse_no,name,pred_ave,rank,reg,odds
9,10,エフフォーリア,17.469374,7.833520,27.105229,2.5
6,7,クロノジェネシス,15.875732,10.199842,21.551622,3.2
4,5,ディープボンド,10.027873,12.072231,7.983515,18.2
8,9,ステラヴェローチェ,9.921543,10.570231,9.272855,9.0
15,16,タイトルホルダー,5.930007,7.833520,4.026493,9.0
1,2,パンサラッサ,5.835410,7.833520,3.837300,32.3
0,1,ペルシアンナイト,5.532409,8.451240,2.613578,51.0
12,13,アカイイト,5.509673,6.077337,4.942009,20.8
5,6,ウインキートス,4.827443,6.291053,3.363832,48.9
3,4,メロディーレーン,4.530800,7.833520,1.228081,71.0


In [528]:
arima_preds.sort_values("pred_ave", ascending=False)

,horse_no,name,pred_ave,rank,reg,odds
9,10,エフフォーリア,17.469374,7.833520,27.105229,2.5
6,7,クロノジェネシス,15.875732,10.199842,21.551622,3.2
4,5,ディープボンド,10.027873,12.072231,7.983515,18.2
8,9,ステラヴェローチェ,9.921543,10.570231,9.272855,9.0
15,16,タイトルホルダー,5.930007,7.833520,4.026493,9.0
1,2,パンサラッサ,5.835410,7.833520,3.837300,32.3
0,1,ペルシアンナイト,5.532409,8.451240,2.613578,51.0
12,13,アカイイト,5.509673,6.077337,4.942009,20.8
5,6,ウインキートス,4.827443,6.291053,3.363832,48.9
3,4,メロディーレーン,4.530800,7.833520,1.228081,71.0


In [358]:
arima_preds.sort_values("reg", ascending=False)

,horse_no,name,pred_ave,rank,reg,odds
9,10,エフフォーリア,17.469374,7.833520,27.105229,2.5
6,7,クロノジェネシス,15.875732,10.199842,21.551622,3.2
8,9,ステラヴェローチェ,9.921543,10.570231,9.272855,9.0
4,5,ディープボンド,10.027873,12.072231,7.983515,18.2
12,13,アカイイト,5.509673,6.077337,4.942009,20.8
10,11,アリストテレス,4.013354,3.954125,4.072583,27.2
15,16,タイトルホルダー,5.930007,7.833520,4.026493,9.0
1,2,パンサラッサ,5.835410,7.833520,3.837300,32.3
5,6,ウインキートス,4.827443,6.291053,3.363832,48.9
0,1,ペルシアンナイト,5.532409,8.451240,2.613578,51.0


In [367]:
pred_ave_rank = df_arima_pred[["horse_no", "pred_ave"]].sort_values("pred_ave", ascending=False).values
pred_ave_rank = {no: p*0.01 for no, p in pred_ave_rank}
pred_ave_rank

{10.0: 0.174693744684384,
 7.0: 0.158757319690739,
 5.0: 0.10027873178856743,
 9.0: 0.09921543360905556,
 16.0: 0.059300065653179364,
 2.0: 0.058354097393139286,
 1.0: 0.055324089858306204,
 13.0: 0.05509672667913373,
 6.0: 0.0482744262813393,
 4.0: 0.045308003536332374,
 11.0: 0.04013354307178681,
 14.0: 0.03377726886388395,
 12.0: 0.02975774245465452,
 15.0: 0.023969830933457344,
 3.0: 0.009748105288192067,
 8.0: 0.008010870213849016}

In [368]:
ave_nums = arima_preds.sort_values("pred_ave", ascending=False)[:5]["horse_no"].values.tolist()
reg_nums = arima_preds.sort_values("reg", ascending=False)[:5]["horse_no"].values.tolist()
rank_nums = arima_preds.sort_values("rank", ascending=False)[:5]["horse_no"].values.tolist()
top5_nums = set(ave_nums + reg_nums + rank_nums)
top5_nums

{1, 2, 5, 7, 9, 10, 13, 16}

In [693]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import chrome
import text
import re
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait


li = re.compile(r"</li>")
tr = re.compile(r"</tr>")

def soup(html):
    html.encoding = html.apparent_encoding
    return BeautifulSoup(html.text, 'html.parser')

def odds_texts(table):
    html_text = table.get_attribute('innerHTML')
    html_text = text.remove_trash(html_text)
    html_text = tr.sub("@" ,html_text)
    html_text = text.td.sub("," ,html_text)
    html_text = li.sub("-" ,html_text)
    html_text = text.tag.sub("", html_text)
    html_text = text.bracket_l.sub("", html_text)
    html_text = text.bracket_r.sub("", html_text)
    results = html_text.split("@")[1:-1]
    results = [row.split(",") for row in results]
    return results

def split_rentan(text):
    return tuple(int(n) for n in text[:-1].split("--"))

def get_oddslist(driver, url):
    time.sleep(1)
    print(f"scraping: {url}")
    driver.get(url)
    if element := chrome.WebDriverWait(driver, timeout=10).until(
        EC.presence_of_all_elements_located
    ):
        tables = driver.find_elements(By.CLASS_NAME, "RaceOdds_HorseList_Table")
        odds_list = odds_texts(tables[0])
        return odds_list
        
race_id = 202106050811
with chrome.driver() as driver:
    odds_url = f"https://race.netkeiba.com/odds/index.html?race_id={race_id}"
    umaren_url = f"https://race.netkeiba.com/odds/index.html?type=b4&race_id={race_id}&housiki=c99"
    umatan_url = f"https://race.netkeiba.com/odds/index.html?type=b6&race_id={race_id}&housiki=c99"
    sanrenpuku_url = f"https://race.netkeiba.com/odds/index.html?type=b7&race_id={race_id}&housiki=c99"
    sanrentan_url = f"https://race.netkeiba.com/odds/index.html?type=b8&race_id={race_id}&housiki=c99"
    tanshou_list = get_oddslist(driver, odds_url)
    tanshou_odds = {int(no): float(odds) for pop, waku, no, _, name, odds, huku, _ in tanshou_list}
    umaren_list = get_oddslist(driver, umaren_url)
    umaren_odds = {split_rentan(niren): float(odds) for pop, _, niren, odds, *_ in umaren_list[1:]}
    umatan_list = get_oddslist(driver, umatan_url)
    umatan_odds = {split_rentan(niren): float(odds) for pop, _, niren, odds, *_ in umatan_list[1:]}
    sanrenpuku_list = get_oddslist(driver, sanrenpuku_url)
    sanrenpuku_odds = {split_rentan(sanren): float(odds) for pop, _, sanren, odds, *_ in sanrenpuku_list[1:]}
    sanrentan_list = get_oddslist(driver, sanrentan_url)
    sanrentan_odds = {split_rentan(sanren): float(odds) for pop, _, sanren, odds, *_ in sanrentan_list[1:]}



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/96.0.4664.45/chromedriver_mac64.zip
Driver has been saved in cache [/Users/chinomirai/.wdm/drivers/chromedriver/mac64/96.0.4664.45]


scraping: https://race.netkeiba.com/odds/index.html?race_id=202106050811
scraping: https://race.netkeiba.com/odds/index.html?type=b4&race_id=202106050811&housiki=c99
scraping: https://race.netkeiba.com/odds/index.html?type=b6&race_id=202106050811&housiki=c99
scraping: https://race.netkeiba.com/odds/index.html?type=b7&race_id=202106050811&housiki=c99
scraping: https://race.netkeiba.com/odds/index.html?type=b8&race_id=202106050811&housiki=c99


In [694]:
def no1_prob(no1_index):
    return pred_ave_rank[no1_index]

def no12_prob(no1_index, no2_index):
    g = 0.81
    no1 = no1_prob(no1_index)
    no2 = pred_ave_rank[no2_index]**g / sum(x**g for x in pred_ave_rank.values())
    return no1, no2, 

def no123_prob(no1_index, no2_index, no3_index):
    d = 0.65
    no1, no2 = no12_prob(no1_index, no2_index)
    no3 = pred_ave_rank[no3_index]**d / sum(x**d for x in pred_ave_rank.values())
    return no1, no2, no3

def sanrentan(a, b, c):
    return a*b*c/((1-a)*(1-b-c))

def sanrenpuku(a, b, c):
    return sanrentan(a,b,c)+sanrentan(a,c,b)+sanrentan(b,a,c)+sanrentan(b,c,a)+sanrentan(c,a,b)+sanrentan(c,b,a)

def umatan(a, b):
    return a*b/(1-b)

def umaren(a, b):
    return umatan(a,b)+umatan(b,a)

import itertools
top5_nums = list(top5_nums)
df_tanshou = pd.DataFrame(
    {
        "一位": [str(no1) for no1 in top5_nums],
        "単勝オッズ(予想)": [0.8/no1_prob(no1) for no1 in top5_nums],
        "単勝オッズ(今)": [tanshou_odds.get(no1) for no1 in top5_nums],
        "単勝確率": [f"{no1_prob(no1)*100:.2f}%" for no1 in top5_nums]
    }
)
umaren_nums = list(itertools.combinations(top5_nums, 2))
df_umaren = pd.DataFrame(
    {
        "一位": [str(no1) for no1, no2 in umaren_nums],
        "二位": [str(no2) for no1, no2 in umaren_nums],
        "馬連オッズ(予想)": [0.775/umaren(*no12_prob(no1, no2)) for no1, no2 in umaren_nums],
        "馬連オッズ(今)": [umaren_odds.get(tuple(sorted([no1, no2]))) for no1, no2 in umaren_nums],
        "馬連確率": [f"{umaren(*no12_prob(no1, no2))*100:.2f}%" for no1, no2 in umaren_nums]
    }
)
umatan_nums = list(itertools.permutations(top5_nums, 2))
df_umatan = pd.DataFrame(
    {
        "一位": [str(no1) for no1, no2 in umatan_nums],
        "二位": [str(no2) for no1, no2 in umatan_nums],
        "馬単オッズ(予想)": [0.775/umatan(*no12_prob(no1, no2)) for no1, no2 in umatan_nums],
        "馬単オッズ(今)": [umatan_odds.get((no1, no2)) for no1, no2 in umatan_nums],
        "馬単確率": [f"{umatan(*no12_prob(no1, no2))*100:.2f}%" for no1, no2 in umatan_nums]
    }
)
sanrenpuku_nums = list(itertools.combinations(top5_nums, 3))
df_sanrenpuku = pd.DataFrame(
    {
        "一位": [str(no1) for no1, no2, no3 in sanrenpuku_nums],
        "二位": [str(no2) for no1, no2, no3 in sanrenpuku_nums],
        "三位": [str(no3) for no1, no2, no3 in sanrenpuku_nums],
        "三連複オッズ(予想)": [0.775/sanrenpuku(*no123_prob(no1, no2, no3)) for no1, no2, no3 in sanrenpuku_nums],
        "三連複オッズ(今)": [sanrenpuku_odds.get(tuple(sorted([no1, no2, no3]))) for no1, no2, no3 in sanrenpuku_nums],
        "三連複確率": [f"{sanrenpuku(*no123_prob(no1, no2, no3))*100:.2f}%" for no1, no2, no3 in sanrenpuku_nums]
    }
)
sanrentan_nums = list(itertools.permutations(top5_nums, 3))
df_sanrentan = pd.DataFrame(
    {
        "一位": [str(no1) for no1, no2, no3 in sanrentan_nums],
        "二位": [str(no2) for no1, no2, no3 in sanrentan_nums],
        "三位": [str(no3) for no1, no2, no3 in sanrentan_nums],
        "三連単オッズ(予想)": [0.775/sanrentan(*no123_prob(no1, no2, no3)) for no1, no2, no3 in sanrentan_nums],
        "三連単オッズ(今)": [sanrentan_odds.get((no1, no2, no3)) for no1, no2, no3 in sanrentan_nums],
        "三連単確率": [f"{sanrentan(*no123_prob(no1, no2, no3))*100:.2f}%" for no1, no2, no3 in sanrentan_nums]
    }
)

In [695]:
print("単勝")
df_tanshou["オッズの歪み"] = df_tanshou["単勝オッズ(今)"] / df_tanshou["単勝オッズ(予想)"]
display(df_tanshou.sort_values("単勝オッズ(予想)", ascending=True))
print("馬連")
df_umaren["オッズの歪み"] = df_umaren["馬連オッズ(今)"] / df_umaren["馬連オッズ(予想)"]
display(df_umaren.sort_values("馬連オッズ(予想)", ascending=True).head(30))
print("馬単")
df_umatan["オッズの歪み"] = df_umatan["馬単オッズ(今)"] / df_umatan["馬単オッズ(予想)"]
display(df_umatan.sort_values("馬単オッズ(予想)", ascending=True).head(30))
print("三連複")
df_sanrenpuku["オッズの歪み"] = df_sanrenpuku["三連複オッズ(今)"] / df_sanrenpuku["三連複オッズ(予想)"]
display(df_sanrenpuku.sort_values("三連複オッズ(予想)", ascending=True).head(30))
print("三連単")
df_sanrentan["オッズの歪み"] = df_sanrentan["三連単オッズ(今)"] / df_sanrentan["三連単オッズ(予想)"]
display(df_sanrentan.sort_values("三連単オッズ(予想)", ascending=True).head(30))

単勝


,一位,単勝オッズ(予想),単勝オッズ(今),単勝確率,オッズの歪み
5,10,4.579443,2.4,17.47%,0.524081
3,7,5.039138,3.3,15.88%,0.654874
2,5,7.977763,18.9,10.03%,2.369085
4,9,8.063262,8.1,9.92%,1.004556
7,16,13.490710,9.1,5.93%,0.674538
1,2,13.709406,33.0,5.84%,2.407107
0,1,14.460247,45.9,5.53%,3.174220
6,13,14.519919,21.5,5.51%,1.480725


馬連


,一位,二位,馬連オッズ(予想),馬連オッズ(今),馬連確率,オッズの歪み
19,7,10,13.798944,3.8,5.62%,0.275383
18,7,9,22.500014,14.0,3.44%,0.622222
15,5,10,22.583845,18.6,3.43%,0.823598
22,9,10,22.838978,8.7,3.39%,0.380928
13,5,7,24.566794,28.3,3.15%,1.151961
26,10,16,31.228509,10.9,2.48%,0.349040
25,10,13,33.204232,29.3,2.33%,0.882418
21,7,16,34.713051,20.5,2.23%,0.590556
14,5,9,36.863112,55.6,2.10%,1.508283
20,7,13,36.909903,42.6,2.10%,1.154162


馬単


,一位,二位,馬単オッズ(予想),馬単オッズ(今),馬単確率,オッズの歪み
38,10,7,27.605603,6.0,2.81%,0.217347
25,7,10,27.747959,7.6,2.79%,0.273894
37,10,5,42.050927,23.1,1.84%,0.549334
39,10,9,42.454066,11.0,1.83%,0.259104
18,5,10,43.929471,56.6,1.76%,1.288429
32,9,10,44.400266,22.9,1.75%,0.515763
23,7,5,46.272096,37.6,1.67%,0.812585
24,7,9,46.715702,20.8,1.66%,0.445246
16,5,7,48.091217,76.0,1.61%,1.580330
31,9,7,48.606613,32.6,1.59%,0.670691


三連複


,一位,二位,三位,三連複オッズ(予想),三連複オッズ(今),三連複確率,オッズの歪み
46,7,9,10,43.070478,7.1,1.80%,0.164846
37,5,7,10,47.501514,15.3,1.63%,0.322095
50,7,10,16,55.606214,10.0,1.39%,0.179836
49,7,10,13,58.539144,22.9,1.32%,0.391191
36,5,7,9,72.090659,48.7,1.08%,0.675538
40,5,9,10,73.415355,31.4,1.06%,0.427703
27,2,7,10,85.874537,51.5,0.90%,0.599712
12,1,7,10,90.901287,43.0,0.85%,0.473041
48,7,9,16,94.208881,35.8,0.82%,0.380007
44,5,10,16,94.764924,48.3,0.82%,0.509682


三連単


,一位,二位,三位,三連単オッズ(予想),三連単オッズ(今),三連単確率,オッズの歪み
230,10,7,5,225.789606,42.7,0.34%,0.189114
231,10,7,9,227.543412,19.8,0.34%,0.087016
152,7,10,5,230.980981,55.7,0.34%,0.241145
153,7,10,9,232.777871,26.6,0.33%,0.114272
224,10,5,7,246.663915,77.4,0.31%,0.313787
237,10,9,7,249.063761,28.1,0.31%,0.112823
141,7,5,10,257.393063,117.4,0.30%,0.456112
147,7,9,10,259.900038,47.3,0.30%,0.181993
110,5,10,7,278.133862,181.6,0.28%,0.652923
195,9,10,7,281.446864,59.4,0.28%,0.211052


In [750]:
import math

def howtobuy(df_tanshou, df_umaren, df_umatan, df_sanrenpuku, df_sanrentan, capital=100, x=1.0, a=0.75, b=0.75, c=0.5, d=0.5):
    ren_return = {}
    tanshou_return = {}
    for i, row in df_tanshou.query(f"{x} < オッズの歪み").iterrows():
        num = math.ceil(capital / row["単勝オッズ(今)"])
        tanshou_return[row['一位']] = (row["単勝オッズ(今)"], num, no1_prob(int(row['一位'])))
    tanshou_nums = tuple(tanshou_return.keys())
    for i, row in df_umaren.query(f"{a} < オッズの歪み and 一位 not in {tanshou_nums}").iterrows():
        num = max(math.ceil(capital / row["馬連オッズ(今)"]), 1)
        a, b = int(row['一位']), int(row['二位'])
        ren_return[f"{row['一位']}-{row['二位']}"] = (row["馬連オッズ(今)"], num, umaren(*no12_prob(a, b)))
    for i, row in df_umatan.query(f"{b} < オッズの歪み and 一位 not in {tanshou_nums}").iterrows():
        num = max(math.ceil(capital / row["馬単オッズ(今)"]), 1)
        a, b = int(row['一位']), int(row['二位'])
        ren_return[f"{row['一位']}->{row['二位']}"] = (row["馬単オッズ(今)"], num, umatan(*no12_prob(a, b)))
    for i, row in df_sanrenpuku.query(f"{c} < オッズの歪み and 一位 not in {tanshou_nums}").iterrows():
        num = max(math.ceil(capital / row["三連複オッズ(今)"]), 1)
        a, b, c = int(row['一位']), int(row['二位']), int(row['三位'])
        ren_return[f"{row['一位']}-{row['二位']}-{row['三位']}"] = (row["三連複オッズ(今)"], num, sanrenpuku(*no123_prob(a, b, c)))
    for i, row in df_sanrentan.query(f"{d} < オッズの歪み and 一位 not in {tanshou_nums}").iterrows():
        num = max(math.ceil(capital / row["三連単オッズ(今)"]), 1)
        a, b, c = int(row['一位']), int(row['二位']), int(row['三位'])
        ren_return[f"{row['一位']}->{row['二位']}->{row['三位']}"] = (row["三連単オッズ(今)"], num, sanrentan(*no123_prob(a, b, c)))
    pred_return = {**tanshou_return, **ren_return}
    return pred_return

def buy(pred_return, cap):
    buy = sum(num for _, num, p in pred_return.values())
    ave_pay = int(sum(odds*num*p for odds, num, p in pred_return.values()))
    min_pay = int(min(odds*num for odds, num, p in pred_return.values()))
    prob = sum(p for odds, num, p in pred_return.values())
    return (ave_pay-buy)*100, (min_pay-buy)*100, buy*100, prob


print("cap cost return prob | x a b c d")
expect_min_pay = 0
expect_ave_pay = 0
expect_ma_pay = -10000
best_prob = 0
avebest_cap = 1
minbest_cap = 1
mabest_cap = 1
prob_cap = 1
for x, a, b, c, d in itertools.product(range(100, 101, 50), range(0, 71, 10), range(0, 101, 10), range(0, 101, 10), range(0, 71, 10)):
    x, a, b, c, d = x/100, a/100, b/100, c/100, d/100
    for cap in range(100, 101, 10):
        pred_buy = howtobuy(df_tanshou, df_umaren, df_umatan, df_sanrenpuku, df_sanrentan, cap, x, a, b, c, d)
        ave_p, min_p, cost, prob = buy(pred_buy, cap)
        print("\r"+f"{cap} {cost} {min_p}-{ave_p} {prob:.2f} | {x} {a} {b} {c} {d} ",end="")
        # if cost <= 10000 and expect_min_pay < min_p:
        #     print("min")
        #     expect_min_pay = min_p
        #     minbest_cap = cap
        #     break
        # if cost <= 10000 and expect_ave_pay < ave_p:
        #     print("ave")
        #     expect_ave_pay = ave_p
        #     avebest_cap = cap
        #     break
        if 9000 <= cost <= 10000 and 0 < min_p and expect_ma_pay < ave_p:
            print("minave")
            expect_ma_pay = ave_p
            mabest_cap = cap
            break
        elif 9000 <= cost <= 10000 and best_prob < prob:
            print("prob")
            best_prob = prob
            prob_cap = cap
            break

cap cost return prob | x a b c d
100 9700 500--3000 0.58 | 1.0 0.0 0.0 0.4 0.6 minave
100 9500 700--2900 0.57 | 1.0 0.0 0.0 0.4 0.7 minave
100 9500 700--2900 0.57 | 1.0 0.0 0.0 0.5 0.6 prob
100 9300 900--2700 0.57 | 1.0 0.0 0.0 0.5 0.7 minave
100 9000 1200--2600 0.55 | 1.0 0.0 0.0 0.8 0.7 minave
100 9700 500--3000 0.58 | 1.0 0.0 0.1 0.4 0.6 5    prob
100 9400 800--2400 0.57 | 1.0 0.3 0.0 0.2 0.4 minave
100 9100 1100--2200 0.55 | 1.0 0.3 0.0 0.5 0.3 minave
100 4000 6500-900 0.42 | 1.0 0.7 1.0 1.0 0.7   1  

In [751]:
pred_buy = howtobuy(df_tanshou, df_umaren, df_umatan, df_sanrenpuku, df_sanrentan, 100, 1.0, 0.0, 0.1, 0.4, 0.6)
display(buy(pred_buy, 100))
display(pred_buy)

(-3000, 500, 9700, 0.5755141857805738)

{'1': (45.9, 3, 0.055324089858306204),
 '2': (33.0, 4, 0.058354097393139286),
 '5': (18.9, 6, 0.10027873178856743),
 '9': (8.1, 13, 0.09921543360905556),
 '13': (21.5, 5, 0.05509672667913373),
 '7-9': (14.0, 8, 0.034444423467526145),
 '7-10': (3.8, 27, 0.05616371752379421),
 '7-13': (42.6, 3, 0.02099707472279321),
 '7-16': (20.5, 5, 0.02232589710466034),
 '10-13': (29.3, 4, 0.02334039835347097),
 '10-16': (10.9, 10, 0.02481706676720654),
 '7-9-13': (77.2, 2, 0.007814871011155322),
 '7-13-16': (111.9, 1, 0.004895905452215138),
 '10-13-16': (66.1, 2, 0.0054969940266498965),
 '7->1->10': (346.4, 1, 0.0017762685284437919),
 '7->5->16': (347.7, 1, 0.0013757507472018798),
 '10->1->7': (253.8, 1, 0.0018543628011242301),
 '10->2->7': (292.7, 1, 0.0019423759461297857)}

In [757]:
pd.DataFrame([(k, v[1]*100)for k, v in pred_buy.items()], columns=["馬券","金額"])

,馬券,金額
0,1,300
1,2,400
2,5,600
3,9,1300
4,13,500
5,7-9,800
6,7-10,2700
7,7-13,300
8,7-16,500
9,10-13,400


In [388]:
arima_preds.sort_values("pred_ave", ascending=False)

,horse_no,name,pred_ave,rank,reg,odds
9,10,エフフォーリア,17.469374,7.833520,27.105229,2.5
6,7,クロノジェネシス,15.875732,10.199842,21.551622,3.2
4,5,ディープボンド,10.027873,12.072231,7.983515,18.2
8,9,ステラヴェローチェ,9.921543,10.570231,9.272855,9.0
15,16,タイトルホルダー,5.930007,7.833520,4.026493,9.0
1,2,パンサラッサ,5.835410,7.833520,3.837300,32.3
0,1,ペルシアンナイト,5.532409,8.451240,2.613578,51.0
12,13,アカイイト,5.509673,6.077337,4.942009,20.8
5,6,ウインキートス,4.827443,6.291053,3.363832,48.9
3,4,メロディーレーン,4.530800,7.833520,1.228081,71.0
